In [7]:
# Cell 1: Imports
import os
import io
from pathlib import Path
from PIL import Image, ImageOps
# --- Use svglib and reportlab instead of cairosvg ---
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPM
# --- End of svglib/reportlab imports ---
import warnings

# Optional: Suppress warnings if needed
# warnings.filterwarnings("ignore", category=UserWarning)

print("Libraries imported (using svglib + reportlab for SVG).")

Libraries imported (using svglib + reportlab for SVG).


In [8]:
# Cell 2: Configuration and Paths (No changes needed here)
INPUT_DIR = Path("MapaPrilezitosti/partners") # <-- Your specific input path
OUTPUT_DIR = Path("processed_logos_ipynb_svglib") # <-- New output dir name to avoid conflicts

CANVAS_WIDTH = 300  # Desired width of the output PNG canvas
CANVAS_HEIGHT = 150 # Desired height of the output PNG canvas
# SVG_DPI is not directly used by svglib in the same way as cairosvg
OUTPUT_FORMAT = "PNG"
BACKGROUND_COLOR = (255, 255, 255, 0) # Transparent RGBA

# Create output directory if it doesn't exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Input directory: {INPUT_DIR.resolve()}")
print(f"Output directory: {OUTPUT_DIR.resolve()}")
print(f"Canvas Size: {CANVAS_WIDTH}x{CANVAS_HEIGHT}px")

Input directory: /Users/omaxim/Documents/GitHub/2ET/MapaPrilezitosti/MapaPrilezitosti/partners
Output directory: /Users/omaxim/Documents/GitHub/2ET/MapaPrilezitosti/processed_logos_ipynb_svglib
Canvas Size: 300x150px


In [9]:
# Cell 3: Preprocessing Function (Modified for svglib)
def preprocess_logo_ipynb_svglib(input_path: Path, output_path: Path, canvas_width: int, canvas_height: int):
    """
    Reads, resizes, centers, and saves a logo onto a transparent canvas.
    Handles PNG directly and SVG using svglib + reportlab. (Notebook version)
    """
    print(f"Processing: {input_path.name} ... ", end="")
    try:
        logo_img = None
        if input_path.suffix.lower() == ".svg":
            # --- Convert SVG using svglib + reportlab ---
            drawing = svg2rlg(str(input_path))
            if drawing is None:
                 raise ValueError("svglib failed to read SVG.")
            # Render the ReportLab drawing to a Pillow Image object
            logo_img = renderPM.drawToPIL(drawing, bg=0xFFFFFF, configOverrides={'transparent': None}) # Use white background for rendering then make transparent if needed later? Or let PIL handle transparency. Check drawtoPIL options.
            # Let's try rendering directly, assuming transparency is handled.
            logo_img = renderPM.drawToPIL(drawing)

            # svglib/reportlab might not preserve transparency as easily as cairosvg
            # If the background isn't transparent, add post-processing if needed
            # For now, assume drawToPIL provides an RGBA image if the SVG had transparency

            if logo_img is None:
                 raise ValueError("renderPM failed to create PIL Image.")
            # --- End of svglib conversion ---

        elif input_path.suffix.lower() == ".png":
            logo_img = Image.open(input_path)
        else:
            print(f"Skipped (unsupported type)")
            return None # Indicate skipped

        # Ensure image is RGBA for transparency handling *after* loading/rendering
        logo_img = logo_img.convert("RGBA")

        # --- Rest of the process is the same ---
        resized_logo = logo_img.copy()
        resized_logo.thumbnail((canvas_width, canvas_height), Image.Resampling.LANCZOS)
        logo_width, logo_height = resized_logo.size

        canvas = Image.new('RGBA', (canvas_width, canvas_height), BACKGROUND_COLOR)
        paste_x = (canvas_width - logo_width) // 2
        paste_y = (canvas_height - logo_height) // 2

        # Paste using the alpha channel of the resized logo as the mask
        canvas.paste(resized_logo, (paste_x, paste_y), resized_logo)

        canvas.save(output_path, format=OUTPUT_FORMAT, optimize=True)
        print(f"Saved: {output_path.name}")
        return True # Indicate success

    except FileNotFoundError:
        print(f"Error: Input file not found!")
        return False # Indicate error
    except Exception as e:
        print(f"Error processing {input_path.name}: {e}")
        # Consider printing traceback for debugging complex errors
        # import traceback
        # traceback.print_exc()
        return False # Indicate error

print("Preprocessing function (svglib version) defined.")

Preprocessing function (svglib version) defined.


In [ ]:
# Cell 4: Execution Loop (Modified to call the new function)
print("-" * 20)
print("Starting preprocessing (using svglib)...")

processed_count = 0
error_count = 0
skipped_count = 0


INPUT_DIR = 'Users/omaxim/Documents/GitHub/2ET/MapaPrilezitosti/MapaPrilezitosti/partners'
# Use glob to find matching files directly
logo_files = list(INPUT_DIR.glob('*.png')) + list(INPUT_DIR.glob('*.svg'))
if not logo_files:
    print(f"\nNo .png or .svg files found in '{INPUT_DIR}'.")
else:
    print(f"Found {len(logo_files)} logos to process.")
    for item_path in logo_files:
        # Construct output path within the specific output directory
        output_file = OUTPUT_DIR / item_path.with_suffix(f".{OUTPUT_FORMAT.lower()}").name
        # Call the svglib version of the function
        result = preprocess_logo_ipynb_svglib(item_path, output_file, CANVAS_WIDTH, CANVAS_HEIGHT) # Removed svg_dpi argument
        if result is True:
            processed_count += 1
        elif result is False:
            error_count += 1
        else: # result is None
            skipped_count += 1
print("-" * 20)
print("Preprocessing complete.")
print(f"Successfully processed: {processed_count}")
print(f"Errors encountered: {error_count}")
print(f"Skipped (unsupported type): {skipped_count}")
print(f"Processed files are in: {OUTPUT_DIR.resolve()}")

--------------------
Starting preprocessing (using svglib)...


AttributeError: 'str' object has no attribute 'is_dir'